#### Initialize

In [1]:
using Revise
using Measurements
using Statistics
using DataFrames
using Plots

# Include modules and additional functions
includet("../../src/LineCableToolbox.jl")
using .LineCableToolbox
Revise.track(LineCableToolbox)

In [ ]:
# VS Code / Jupyter string formatting
struct PlainString
    str::String
end

Base.show(io::IO, ::MIME"text/plain", ps::PlainString) = print(io, ps.str)

# Initialize plotlyjs backend
plotlyjs()  
plot(size=(0,0), [0, 0], [0, 0], seriestype=:scatter, markersize=0, 
     title="", xlabel="x-axis", ylabel="y-axis", legend=false)

#### Materials library

In [ ]:
materials_db = MaterialsLibrary()
display_materials_library(materials_db)

#### Process measurements with uncertainties

In [ ]:
# Process measurements to obtain the main components dimensions

# Core
d_wire_nominal = 4.67e-3  
d_core_nominal = 38.1e-3
m1 = measurement(0.03761, 0.00014)
m2 = measurement(0.03776, 0.00014)
d_core_meas = mean([m1, m2])
d_core_meas = bias_to_uncertain(d_core_nominal, [m1, m2])
d_w = bias_to_uncertain(d_wire_nominal, [d_core_meas/9])

# Inner semicon
m1 = measurement(0.000755, 5.5e-5)
m2 = measurement(0.00078, 5.0e-5)
t_sc_in = mean([m1, m2])

# Main insulation
m1 = measurement(0.00825, 0.00014)
m2 = measurement(0.00836, 0.00022)

t_ins = mean([m1, m2])

# Outer semicon
m1 = measurement(0.000475, 4.5e-5)
m2 = measurement(0.00047, 4.0e-5)
t_sc_out = mean([m1, m2])

# Total sheath thickness inferred from QA measurements (inner pe_sheath - outer semicon_out) must fit the core wbt + wire screen + copper tape + wbt + aluminum tape
m1 = measurement(0.05891,0.00065)-measurement(0.05676, 0.00021)
m2 = measurement(0.05902,0.00037)-measurement(0.05721, 0.00023)
sheath_thickness_meas = mean([m1, m2])

d_wsc_nominal = .94e-3  # nominal wire screen diameter
t_cut_nominal = 0.1e-3 # nominal thickness of the copper tape
t_wbt_nominal = .3e-3 # nominal thickness of the water blocking tape
t_alt_nominal = .15e-3 # nominal thickness of the aluminum tape
t_pet_nominal = .05e-3 # nominal thickness of the pe face in the aluminum tape
@show  sheath_thickness_nom = t_wbt_nominal + d_wsc_nominal + t_cut_nominal + t_wbt_nominal + t_alt_nominal + t_pet_nominal

# Incorporate the difference between nominal and measured into uncertainty
@show  sheath_thickness_meas = bias_to_uncertain(sheath_thickness_nom, [m1, m2])

# Adjust the nominal dimensions to the measured sheath thickness
@show d_ws = d_wsc_nominal * sheath_thickness_meas / sheath_thickness_nom
@show t_cut = t_cut_nominal * sheath_thickness_meas / sheath_thickness_nom
@show t_wbt = t_wbt_nominal * sheath_thickness_meas / sheath_thickness_nom
@show t_alt = t_alt_nominal * sheath_thickness_meas / sheath_thickness_nom
@show t_pet = t_pet_nominal * sheath_thickness_meas / sheath_thickness_nom

# PE jacket thickness
m1 = measurement(0.00328, 0.00028)
m2 = measurement(0.0034, 0.00033) 
@show t_jac = mean([m1, m2])
;

In [ ]:
@show (d_ws+t_cut+t_wbt+t_alt+t_pet)
@show _percent_error(d_ws)
@show _percent_error(sheath_thickness_meas)
@show _percent_error(t_cut)
@show _percent_error(t_wbt)
@show _percent_error(t_alt)

# note to future self: differences reduce the deterministic value and increase the absolute uncertainty because **errors add** even in subtraction, causing the error to grow out of proportion
@show _percent_error(measurement(0.05891,0.00065))
@show _percent_error(measurement(0.05676,0.00021))
@show _percent_error(measurement(0.05902,0.00037)-measurement(0.05721, 0.00023))
;

In [ ]:
#= Using tolerances defined in  60502-1 © IEC:2004+A1:2009 (p. 27)
16.7.3 Requirements
The dimensions of armour wires and tapes shall not fall below the nominal values given in 12.5 # by more than:
– 5 % for round wires;
– 8 % for flat wires;
– 10 % for tapes. =#
# @show d_ws = percent_to_uncertain(d_wsc_nominal,5)
# @show t_cut = percent_to_uncertain(t_cut_nominal,10)
# @show t_alt = percent_to_uncertain(t_alt_nominal, 10)
# @show t_wbt = percent_to_uncertain(t_wbt_nominal,20)
# @show t_pet = percent_to_uncertain(t_pet_nominal, 20)

#### Cable core

In [ ]:
# Stranded conductor core
material = get_material(materials_db, "aluminum")
core = Conductor(WireArray(0, @diam(d_w), 1, 0, material))
add_conductor_part!(core, WireArray, @diam(d_w), 6, 15, material)
add_conductor_part!(core, WireArray, @diam(d_w), 12, 13.5, material)
add_conductor_part!(core, WireArray, @diam(d_w), 18, 12.5, material)
add_conductor_part!(core, WireArray, @diam(d_w), 24, 11, material)

# Inner semiconductive tape (optional)


# Inner semiconductor
material = get_material(materials_db, "semicon1")
semicon_in = Semicon(core, @thick(t_sc_in), material)

# Main insulation
material = get_material(materials_db, "xlpe")
main_insu = Insulator(semicon_in, @thick(t_ins), material)

# Outer semiconductor
material = get_material(materials_db, "semicon2")
semicon_out = Semicon(main_insu, @thick(t_sc_out), material)

# Core semiconductive tape
material = get_material(materials_db, "polyacrylate")
wb_tape_co = Semicon(semicon_out, @thick(t_wbt), material)

core_parts = [core, semicon_in, main_insu, semicon_out, wb_tape_co]

# Initialize CableDesign with the first component
datasheet_info = NominalData(
    conductor_cross_section = 1000.0,   # [mm²]
    screen_cross_section = 35.0,        # [mm²]
    resistance = 0.0291,                # [Ω/km]
    capacitance = 0.39,                 # [μF/km]
    inductance = 0.3                    # [mH/km]
)

cable_design = CableDesign("NA2XS(FL)2Y 18/30", "core", core_parts, nominal_data = datasheet_info)

#### Wire screens

In [ ]:
num_wires = 49
lay_ratio = 10
w_cut = 10e-3 # width of copper tape

# Continue building on top of wb_tape_co
material = get_material(materials_db, "copper")
wire_screen = Conductor(WireArray(wb_tape_co, @diam(d_ws), num_wires, lay_ratio, material))
add_conductor_part!(wire_screen, Strip, @thick(t_cut), w_cut, lay_ratio, material)
@show wire_screen.resistance
@show _percent_error(wire_screen.resistance)

# Water blocking tape
material = get_material(materials_db, "polyacrylate")
wb_tape_scr = Semicon(wire_screen, @thick(t_wbt), material)

sheath_parts = [wire_screen, wb_tape_scr]
add_cable_component!(cable_design, "sheath", sheath_parts)
;

#### PE Jacket

In [ ]:
# Continue building on top of wb_tape_scr
material = get_material(materials_db, "aluminum")
alu_tape = Conductor(Tubular(wb_tape_scr,  @thick(t_alt), material))

# PE layer after aluminum foil 
material = get_material(materials_db, "pe")
alu_tape_pe = Insulator(alu_tape, @thick(t_pet_nominal), material)

# PE jacket
material = get_material(materials_db, "xlpe")
pe_insu = Insulator(alu_tape_pe, @thick(t_jac), material)

jacket_parts = [alu_tape, alu_tape_pe, pe_insu]
add_cable_component!(cable_design, "jacket", jacket_parts)
;

#### Preview

In [ ]:
display(PlainString("- Comparison with datasheet information:"))
display(core_parameters(cable_design))

display(PlainString("- EMT parameters:"))
display(cable_data(cable_design))

display(PlainString("- Detailed description of cable parts:"))
display(cable_parts_data(cable_design))

preview_cable_cross_section(cable_design)

#### Save to cables library

In [ ]:
library = CablesLibrary()
display_cables_library(library)
add_cable_design!(library, cable_design)
save_cables_library(library)

#### Frequency-dependent earth model

In [ ]:
f = 10.0 .^ range(0, stop=6, length=10)
earth_params = EarthModel(f, 100, 10, 1)
# earth_params.EHEMformulation=EnforceLayer(-1)
# add_earth_layer!(earth_params, 200, 15, 1, t=Inf)
earth_params

EarthModel(3, ConstantProperties("CP model"), EnforceLayer(-1, "Bottom layer"), false, EarthLayer[EarthLayer(Inf, 1.0, 1.0, Inf, Number[Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf], Number[8.8541878128e-12, 8.8541878128e-12, 8.8541878128e-12, 8.8541878128e-12, 8.8541878128e-12, 8.8541878128e-12, 8.8541878128e-12, 8.8541878128e-12, 8.8541878128e-12, 8.8541878128e-12], Number[1.2566370614359173e-6, 1.2566370614359173e-6, 1.2566370614359173e-6, 1.2566370614359173e-6, 1.2566370614359173e-6, 1.2566370614359173e-6, 1.2566370614359173e-6, 1.2566370614359173e-6, 1.2566370614359173e-6, 1.2566370614359173e-6]), EarthLayer(100, 10, 1, 5, Number[100, 100, 100, 100, 100, 100, 100, 100, 100, 100], Number[8.8541878128e-11, 8.8541878128e-11, 8.8541878128e-11, 8.8541878128e-11, 8.8541878128e-11, 8.8541878128e-11, 8.8541878128e-11, 8.8541878128e-11, 8.8541878128e-11, 8.8541878128e-11], Number[1.2566370614359173e-6, 1.2566370614359173e-6, 1.2566370614359173e-6, 1.2566370614359173e-6, 1.2566370614359